# Overview

This interactive notebook acts as an productive, ituitive way for the Speckle Imaging Group to plan observing runs. In order to ensure that this tool functions properly, there are a list of **assumptions**:

- Input files will be CSV files aka comma seperated files (create a fork for it to be tab separated)
- For complex filtering and searching of targets, you must provide your own custom filter in the cells marked below if the ones provided are not suitable
- For "ZERO" values or "EMPTY" values, please use empty commas ",,"

## Download libraries if not 

Depending on your set up, you may have to change "pip3" to "pip"

In [ ]:
!pip3 install --upgrade pip
!pip3 install pandas numpy
!pip3 install astropy astroplan -U
!pip3 install ipywidgets

## Set Up Libraries and Constants

In [2]:
from astropy.coordinates import SkyCoord
import astropy
import ipywidgets as widgets
from IPython.display import display, clear_output
from astropy.table import QTable
import astropy.units as u
import numpy as np
import pandas as pd

import ipywidgets as widgets
observed_textbox = widgets.Textarea(
    value='''Example:
1
2
10
15
''',
    placeholder='Type something',
    description='Observed: ',
    disabled=False
)

# Step 1: Importation of olist/target files 

Note: There are two different "Step 1"s.

[Step 1a is intended for a first night of an observing run](#Step-1a:-ASCII-File-Importation)

[Step 1b is intended as a continuation of a previous night's run](#Step-1b:-Resume-Observation-Run-From-Previous-Night(s))

## Step 1a: ASCII File Importation 

### Upload CSV File via Upload Widget

**Columns:**

column 1: star_id
column 2: queue_id
column 3: right ascension
column 4: declination
column 5: epoch
column 6: proper(
column 7: proper(dec)
column 8: magnitude
column 9: priority
column 10: principal investigator
column 11: comment

**Example Input file:**

HIP33451,06:57:17.6,-35:30:26,2000.00,,,6.22,3,joncwong,comment<br/> 
HR 2518,06:47:21.4,-37:55:47,2000.00,-7.0,-15.0,5.26,1,howell<br/> 
HIP38382,07:51:46.3,-13:53:53,2000.00,,,5.16,1,nic,testtesttesttest<br/> 
HR 3073,07:52:18.9,-14:50:47,2000.00,-11.0,-2.0,5.69,1,nic<br/> 
HIP41426,08:26:57.7,-52:42:18,2000.00,,,6.49,2,howell,another test comment<br/> 
HR 3350,08:27:36.5,-53:05:19,2000.00,-68.0,13.0,5.09,,,<br/> 
HIP78727,16:04:22.2,-11:22:23,2000.00,,,4.17,,,test comment<br/> 
HR 6048,16:13:50.9,-11:50:15,2000.00,-10.0,-6.0,5.22,,,test comment<br/> 
H101769,20:37:32.9,+14:35:43,2000.00,,,4.03,,,<br/> 
HR 7918,20:41:16.2,+14:34:59,2000.00,-6.0,2.0,5.99,,,<br/> 
H104858,21:14:28.8,+10:00:28,2000.00,,,5.07,,,<br/> 
HR 8149,21:18:52.0,+11:12:12,2000.00,26.0,15.0,5.96,,,<br/> 
H111974,22:40:52.5,+14:32:56,2000.00,,,6.14,,,<br/> 
HR 8642,22:41:57.4,+14:30:59,2000.00,92.0,-21.0,5.9,,,COMMENT

In [ ]:
uploaded = widgets.FileUpload(
    accept='',
    multiple=True
)
uploaded

### Step 1a cont: Parse CSV File into a Python List

We parse the CSV File into a List data structure before importing the list into an AstroPy table

In [5]:
decoded_files = []
for file, file_bytes in uploaded.value.items():
    decoded_files.append(file_bytes['content'].decode())

decoded_entries = []

for decoded_file in decoded_files:
    for line in decoded_file.split("\n"):
        line = ' '.join(line.split())
        if line.count(",") > 2:
            line = line.split(",")
        else:
            line = line.split(" ")
        
        if line[0] == "":
            continue
   
        try:
            if line[4] == '':
                line[4] = 0.0
        except:
            line.append(0.0)

        try:
            if line[5] == '':
                line[5] = 0.0
        except:
            line.append(0.0)
        try:
            if line[6] == '':
                line[6] = 0.0
        except:
            line.append(0.0)

        try:
            comment = line[9]
        except:
            line.append("")
        decoded_entries.append(line)

[['HIP33451', '06:57:17.6', '-35:30:26', '2000.00', 0.0, 0.0, '6.22', '3', 'joncwong', 'comment'], ['HR 2518', '06:47:21.4', '-37:55:47', '2000.00', '-7.0', '-15.0', '1', 'howell', '5.26', ''], ['HIP38382', '07:51:46.3', '-13:53:53', '2000.00', 0.0, 0.0, '5.16', '1', 'nic', 'testtesttesttest'], ['HR 3073', '07:52:18.9', '-14:50:47', '2000.00', '-11.0', '-2.0', '5.69', '1', 'nic', ''], ['HIP41426', '08:26:57.7', '-52:42:18', '2000.00', 0.0, 0.0, '6.49', '2', 'howell', 'another test comment'], ['HR 3350', '08:27:36.5', '-53:05:19', '2000.00', '-68.0', '13.0', '5.09', '', '', ''], ['HIP78727', '16:04:22.2', '-11:22:23', '2000.00', 0.0, 0.0, '4.17', '', '', 'test comment 2'], ['HR 6048', '16:13:50.9', '-11:50:15', '2000.00', '-10.0', '-6.0', '5.22', '', '', 'test comment'], ['H101769', '20:37:32.9', '+14:35:43', '2000.00', 0.0, 0.0, '4.03', '', '', ''], ['HR 7918', '20:41:16.2', '+14:34:59', '2000.00', '-6.0', '2.0', '5.99', '', '', ''], ['H104858', '21:14:28.8', '+10:00:28', '2000.00', 0.

# Backlog Table and Observed Table Construction

In this cell, we created two astropy tables: **backlog_table** and **observed_table**

"backlog_table" will hold all the targets that we have yet to observed. "observed_table" will hold all the targets have have been observed.


In [6]:
backlog_targets = []
observed_targets = []

# Construction of Backlog table
# backlog_table = QTable(names=("star_id", "queue_id", "ra","dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
backlog_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
backlog_table["star_id"].dtype = np.dtype("S10")
# backlog_table["queue_id"].dtype = np.dtype("S10")
backlog_table["ra"].dtype = np.dtype("S10")
backlog_table["dec"].dtype = np.dtype("S10")
backlog_table["epoch"].dtype = np.dtype("float32")
backlog_table["proper(ra)"].dtype = np.dtype("float32")
backlog_table["proper(dec)"].dtype = np.dtype("float32")
backlog_table["mag"].dtype = np.dtype("float32")
backlog_table["priority"].dtype = np.dtype("float32")
backlog_table["pi"].dtype = np.dtype("S10")
backlog_table["comment"].dtype = np.dtype("S10")

# Construction of Observed table
# observed_table = QTable(names=("star_id", "queue_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
observed_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
observed_table["star_id"].dtype = np.dtype("S10")
# observed_table["queue_id"].dtype = np.dtype("S10")
observed_table["ra"].dtype = np.dtype("S10")
observed_table["dec"].dtype = np.dtype("S10")
observed_table["epoch"].dtype = np.dtype("float32")
observed_table["proper(ra)"].dtype = np.dtype("float32")
observed_table["proper(dec)"].dtype = np.dtype("float32")
observed_table["mag"].dtype = np.dtype("float32")
observed_table["priority"].dtype = np.dtype("float32")
observed_table["pi"].dtype = np.dtype("S10")
observed_table["comment"].dtype = np.dtype("S10")

# Insert data into the table
for i in range(len(decoded_entries)):
    backlog_table.insert_row(i, decoded_entries[i])

ValueError: Unable to insert row because of exception in column 'priority':
could not convert string to float: 'howell'

## Step 1b: Resume Observation Run From Previous Night(s)

TODO: Allow prompted files to be inputted rather than a hard coded absolute file path

TODO: Make instructions for how to do file picker

In [4]:
backlog_table = QTable.read('backlog_table.ecsv')
observed_table = QTable.read('observed_table.ecsv')

# Parameter Tuning

In [5]:
# Try using a enterable box for these parameters
global ra_lower_bound
global ra_upper_bound
global dec_lower_bound
global dec_upper_bound

# Edit these parameters
ra_lower_bound = '00:00:00.0'
ra_upper_bound = '24:00:00.0'
dec_lower_bound = '-90:00:00.0'
dec_upper_bound = '+90:00:00.0'
allowed_priority_bands = []

# Range filter for mag
# Display comments on the table
# Filter by bands as well

# def filter_ra_by_range(table, column):
#     if ra_lower_bound < table[column][] < ra_upper_bound:
#         return True
#     return False

# Edit the parameters of group_by in order to sort by columns of your choice
# Possible sort keys: idx, star_id, ra, dec, epoch, proper(ra), proper(dec), mag
backlog_table = backlog_table.group_by(['ra', 'mag'])

# backlog_table = backlog_table.groups.filter(filter_ra_by_range)
# backlog_table = backlog_table.groups.filter(filter_dec_by_range)

# Table Display

- If a different range, filter, or sort by is needed, rerun "Parameter Tuning" code block
    - Then, re-run "Table Display"

In [6]:
backlog_table.show_in_notebook(display_length=25)

idx,star_id,ra,dec,epoch,proper(ra),proper(dec),mag,comment
0,HR 2518,06:47:21.4,-37:55:47,2000.0,-7.0,-15.0,5.26,--
1,HIP38382,07:51:46.3,-13:53:53,2000.0,0.0,0.0,5.16,testtestte
2,HR 3073,07:52:18.9,-14:50:47,2000.0,-11.0,-2.0,5.69,--
3,HIP41426,08:26:57.7,-52:42:18,2000.0,0.0,0.0,6.49,another te
4,HR 6048,16:13:50.9,-11:50:15,2000.0,-10.0,-6.0,5.22,test comme
5,H101769,20:37:32.9,+14:35:43,2000.0,0.0,0.0,4.03,--
6,HR 7918,20:41:16.2,+14:34:59,2000.0,-6.0,2.0,5.99,--
7,H104858,21:14:28.8,+10:00:28,2000.0,0.0,0.0,5.07,--
8,HR 8149,21:18:52.0,+11:12:12,2000.0,26.0,15.0,5.96,--
9,H111974,22:40:52.5,+14:32:56,2000.0,0.0,0.0,6.14,--


# Mark as Observed

Mark stars as observed in the below textbox. Remove the example placeholder shown with your own list of 'idx's to mark as observed.

After 

TODO: 

allow for comments after the ID

In [9]:
observed_textbox

Textarea(value='Example:\n1\n2\n10\n15\n', description='Observed: ', placeholder='Type something')

## Move targets from textbox to saved observed table

Run the below cell in order to save all of the targets by idx, into a temporary table which will be exported at the end of the night.

**Note:**

- If an entry is not a number or a valid ID of the table, it will be automatically thrown away.**

- Idx numbers are not permanent nor important to a particular row, they are simply used as temporary IDs to mark targets as observed

In [8]:
textbox_string = observed_textbox.value
textbox_string = textbox_string.strip()
textbox_ids = textbox_string.split('\n')
id_upper_bound = len(backlog_table)
textbox_ids = [s for s in textbox_ids if s.isdigit()]
textbox_ids = map(int, textbox_ids)
textbox_ids = [s for s in textbox_ids if s < id_upper_bound]

for idx in textbox_ids:
    observed_table.add_row(backlog_table[idx])

backlog_table.remove_rows(textbox_ids)

In [7]:
observed_table.show_in_notebook(display_length=25)

idx,star_id,ra,dec,epoch,proper(ra),proper(dec),mag,comment
0,HIP33451,06:57:17.6,-35:30:26,2000.0,0.0,0.0,6.22,comment
1,HR 3350,08:27:36.5,-53:05:19,2000.0,-68.0,13.0,5.09,--
2,HIP78727,16:04:22.2,-11:22:23,2000.0,0.0,0.0,4.17,test comme


# Undo/Redo

Use these boilerplate cells to manually undo/redo addition and removal of rows between tables

TODO: "ID 1 from backlog_table to observed_table, vice versa"

Replicate observed_textbox with no comments

In [8]:
# Example boilerplate to manually add back a row to backlog_table
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])

ValueError: Mismatch between number of vals and columns

In [ ]:
# Example boilerplate to manually add back a row to observed_table
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])

In [ ]:
backlog_table.remove_rows([1, 3, 100, 111])

In [ ]:
observed_table.remove_rows([1, 3, 100, 111])

# Save table state(s) for the night

In order to save the state(s) of the table, we will be serializing both the backlog and observed table. When loading data for the next night, only these saved files will be needed, not the initial target files. Will overr



In [10]:
import datetime

backlog_table.write('backlog_table_' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')
observed_table.write('observed_table' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')